In [1]:
from crossreference import *
import chart.johncartersqueeze as jcsqueeze

In [2]:
import pandas as pd
import numpy as np
import datetime

In [3]:
import mysql.connector

def load_stockdata(symbol):
    serverName = "localhost"
    cnx = mysql.connector.connect(user='michael', password='Password2017',
                                  host=serverName, database='stock_market')
    cursor = cnx.cursor()
    
    query = \
    "SELECT stock.*, bb.`sma` AS bb_middle, bb.`band_upper` AS bb_upper, bb.`band_lower` AS bb_lower, \
    kc.`ema20` AS kc_middle, kc.`atr2_upper` AS kc_2_upper, kc.`atr2_lower` AS kc_2_lower, \
    kc.`atr3_upper` AS kc_3_upper, kc.`atr3_lower` AS kc_3_lower, \
    kc.`atr1_upper` AS kc_1_upper, kc.`atr1_lower` AS kc_1_lower, \
    macd.`macd_histogram` as macd, ttma.macd_histogram as wave_a, ttmb.macd_histogram as wave_b, ttmc.macd_histogram as wave_c \
    FROM \
    ( SELECT * FROM (SELECT symbol, effective_date, `adj_open`, `adj_high`, `adj_low`, `adj_close`, `adj_volume` FROM `market.stock_price` \
      WHERE symbol = '{0}' ORDER BY effective_date DESC LIMIT 180) sub ORDER BY `effective_date` ASC ) stock \
    LEFT JOIN \
    `idc.bollinger_bands` AS bb ON bb.symbol = stock.symbol AND bb.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.keltner_channels` AS kc ON kc.symbol = stock.symbol AND kc.effective_date = stock.effective_date \
    LEFT JOIN \
    `idc.macd` as macd on macd.symbol = stock.symbol and macd.effective_date = stock.effective_date \
        and macd.short_period = 12 and macd.long_period=26 and macd.signal_period=9 \
    LEFT JOIN \
    `idc.macd` as ttma on ttma.symbol = stock.symbol and ttma.effective_date = stock.effective_date \
        and ttma.short_period = 8 and ttma.long_period=34 and ttma.signal_period=34 \
    LEFT JOIN \
    `idc.macd` as ttmb on ttmb.symbol = stock.symbol and ttmb.effective_date = stock.effective_date \
        and ttmb.short_period = 8 and ttmb.long_period=89 and ttmb.signal_period=89 \
    LEFT JOIN \
    `idc.macd` as ttmc on ttmc.symbol = stock.symbol and ttmc.effective_date = stock.effective_date \
        and ttmc.short_period = 8 and ttmc.long_period=144 and ttmc.signal_period=144 \
    ".format(symbol)

    cursor.execute(query)
    dfsql = pd.DataFrame(cursor.fetchall(), dtype=np.float)
    dfsql.columns = cursor.column_names
    dfsql.set_index('effective_date', inplace=True)
    cursor.close()
    return dfsql


In [11]:
df = load_stockdata('ALB')
# df['macd_up'] = df.macd > df.macd.shift(-1)
#df['squeeze'] = (df.bb_lower >= df.kc_2_lower) & (df.bb_upper <= df.kc_2_upper)
df.head(20)


,symbol,adj_open,adj_high,adj_low,adj_close,adj_volume,bb_middle,bb_upper,bb_lower,kc_middle,kc_2_upper,kc_2_lower,kc_3_upper,kc_3_lower,kc_1_upper,kc_1_lower,macd,wave_a,wave_b,wave_c
effective_date,,,,,,,,,,,,,,,,,,,,
2018-02-23,ALB,115.84,118.8300,115.610,118.75,2136021.0,110.7305,117.948046,103.512954,114.322225,122.308841,106.335609,126.302149,102.342301,118.315533,110.328917,1.512123,NaN,NaN,NaN
2018-02-22,ALB,113.88,114.7499,113.155,113.68,1021914.0,110.6380,117.479698,103.796302,113.856143,121.664805,106.047481,125.569136,102.143150,117.760474,109.951812,1.124417,NaN,NaN,NaN
2018-02-21,ALB,115.35,116.5000,112.870,113.01,1765680.0,110.5740,117.324008,103.823992,113.874684,122.016336,105.733032,126.087162,101.662206,117.945510,109.803858,0.992081,NaN,NaN,NaN
2018-02-20,ALB,112.00,115.3900,112.000,114.58,1420025.0,110.5735,117.322788,103.824212,113.965704,122.175176,105.756232,126.279912,101.651496,118.070440,109.860968,0.834333,NaN,NaN,NaN
2018-02-16,ALB,114.36,114.5900,111.960,112.24,1668940.0,110.5210,117.160250,103.881750,113.901041,122.220471,105.581611,126.380186,101.421896,118.060756,109.741326,0.424773,NaN,NaN,NaN
2018-02-15,ALB,112.43,114.6500,111.380,114.61,1746847.0,110.7095,117.736046,103.682954,114.075887,122.627581,105.524193,126.903428,101.248346,118.351734,109.800040,0.048849,0.016886,-11.484235,-17.977815
2018-02-14,ALB,107.18,112.1600,106.630,111.72,2051713.0,110.8745,118.397140,103.351860,114.019665,122.726105,105.313225,127.079325,100.960005,118.372885,109.666445,-0.719983,-1.263551,-13.092772,-19.493402
2018-02-13,ALB,111.60,112.6000,108.084,108.45,2756593.0,111.6360,121.922492,101.349508,114.261735,122.787133,105.736337,127.049832,101.473638,118.524434,109.999036,-1.515640,-2.309295,-14.312913,-20.577146
2018-02-12,ALB,109.28,112.1500,109.010,111.59,2636101.0,112.6135,125.003652,100.223348,114.873496,123.359924,106.387068,127.603138,102.143854,119.116710,110.630282,-0.125272,-2.890232,-14.896071,-20.978917


In [12]:
jcsqueeze.drawchart('MSFT', df)

In [6]:
import repository.stockpricerepo as pricerepo

In [8]:
df = pricerepo.get_stock_adjdata('ALB', datetime.date(2017,1,1), datetime.date(2018,2,20))
df.head()

,adj_open,adj_high,adj_low,adj_close,adj_volume
effective_date,,,,,
2017-01-03,86.418382,87.375281,86.249809,87.310827,975594.0
2017-01-04,87.975202,90.384802,87.945454,89.938580,1147617.0
2017-01-05,89.879084,90.622788,88.976723,89.631183,913202.0
2017-01-06,89.859252,90.236062,89.105632,90.047657,550924.0
2017-01-09,89.968328,90.216229,89.611350,89.720427,936082.0


In [6]:
import repository.simpleindicatorrepo as pricerepo

ImportError: cannot import name 'indicatorrepo'